# AI OCR:
This file follows the idea from the paper "MULTIMODAL LLMS FOR OCR, OCR POST-CORRECTION, AND NAMED ENTITY RECOGNITION IN HISTORICAL DOCUMENTS" from Gavin Greif, Niclas Griesshaber and Robin Greif.

Using the same prompt from the paper as template and styling it to our project needs I feed cropped images to the ai model, which returns a json file.
Then a second prompt is used to enforce the desired format of the final text. The result is a csv file with addresses.

The addresses which found are later golocated using geopy free open source geocoding from OpenStreetMap: Nominatim.

GIT - https://github.com/niclasgriesshaber/gemini_historical_dataset_pipeline/tree/main


In [ ]:
import pandas as pd
import numpy as np
import re
import os
from os import path
from PIL import Image
import cv2
import google.generativeai as genai
# from google import genai
import csv
import io
from matplotlib import pyplot as plt
import json
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
# mount google colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Include here your directory
%cd /content/drive/MyDrive/2_FEINA/10_HU/01_TASKS/01_DIRECTORIES


/content/drive/MyDrive/2_FEINA/10_HU/01_TASKS/01_DIRECTORIES


In [ ]:
# Set Current Directory:
#
cwd = os.getcwd()

# AI_FILES folder should contain all cities. Create folders for all msa cities
cities = pd.read_excel('City_list_coreUS.xlsx')
city_list = []
for element in cities['metro']:
  city_list.append(element)

ai_files_dir = os.path.join(cwd, "ai_files")
os.makedirs(ai_files_dir, exist_ok=True)

for element in city_list:
  if path.exists(cwd + '/ai_files/' + element) == False:
    os.mkdir(cwd + '/ai_files/' + element)

## OCR

Using AI to read and recognize the text I feed the model coming from a series of pre_processed pages.
The pages are in a folder called "temp", inside of which there is a folder for each city and inside each city there are all the processed pages.





In [ ]:
###############################################################################
# Project Paths
###############################################################################
city = "Kansas City"
state = 'Missouri'
country = "USA"
###############################################################################
# Used already pre-processed images stored in a temp folder
processed_directory = os.path.join(cwd + f"/temp/{city}")

filenames = os.listdir(processed_directory)
print(filenames) # many files, will have to filter later

['page_1.jpg', 'contrast_enhanced_page_page_1.jpg.jpg', 'denoised_page_page_1.jpg.jpg', 'pre_processed_page_page_1.jpg.jpg', 'blur_page_page_1.jpg.jpg', 'thresh_page_page_1.jpg.jpg', 'binary_page_page_1.jpg.jpg', 'gray_page_page_1.jpg.jpg', 'sharpened_page_page_1.jpg.jpg', 'ny_bbox_page_page_1.jpg.jpg', 'dilate_page_page_1.jpg.jpg', 'page_2.jpg', 'contrast_enhanced_page_page_2.jpg.jpg', 'denoised_page_page_2.jpg.jpg', 'binary_page_page_2.jpg.jpg', 'pre_processed_page_page_2.jpg.jpg', 'sharpened_page_page_2.jpg.jpg', 'gray_page_page_2.jpg.jpg', 'blur_page_page_2.jpg.jpg', 'dilate_page_page_2.jpg.jpg', 'thresh_page_page_2.jpg.jpg', 'ny_bbox_page_page_2.jpg.jpg', 'page_3.jpg', 'contrast_enhanced_page_page_3.jpg.jpg', 'denoised_page_page_3.jpg.jpg', 'sharpened_page_page_3.jpg.jpg', 'binary_page_page_3.jpg.jpg', 'pre_processed_page_page_3.jpg.jpg', 'blur_page_page_3.jpg.jpg', 'gray_page_page_3.jpg.jpg', 'thresh_page_page_3.jpg.jpg', 'dilate_page_page_3.jpg.jpg', 'ny_bbox_page_page_3.jpg.jpg

### Rreading prompt:
The prompt I use is the same from the above mentioned paper, but imposing the following fields and forms of those:

* City (string)
*  Company Name or Owners's Name (string)  
* Company Type" (string, ONLY OPTIONS : 'Attorney:', 'Real Estate:', 'Land Companies:', "Abstract Companies:","Abstractors:","Abstracts:", "Land Script:")
* "building" (string; if the address is a building add it here too)   
*  "address" (string; full address if possible, otherwise partial)


The rules in the prompt are based on what the authors designed, some new rules were added to fit our needs in the directories setup. The full prompt can be read in the next cell

In [ ]:
# READING THE PROMP:

prompt_file_path = os.path.join(cwd, "prompt.txt")
# Read the prompt from the file
def read_prompt(prompt_file_path):
  try:
      with open(prompt_file_path, "r") as f:
          prompt = f.read().strip()
      print(f"Loaded prompt from: {prompt_file_path}")
      print(f"Prompt content: {prompt}")
  except FileNotFoundError:
      print(f"Error: Prompt file not found at {prompt_file_path}")
      exit()
  return prompt

prompt = read_prompt(prompt_file_path)

Loaded prompt from: /content/drive/MyDrive/2_FEINA/10_HU/01_TASKS/01_DIRECTORIES/prompt.txt
Prompt content: YOU ARE AN EXPERT HISTORIAN. YOUR TASK IS TO EXTRACT DATA FROM A SCAN OF A US BUSINESS DIRECTORY. PLEASE FOLLOW THESE FORMATTING RULES STRICTLY. YOUR GOAL IS TO MAXIMIZE ENTRY POINTS WITH CORRECT DATA. MISSING OR INVALID FORMATS MAY LEAD TO EXCLUSION OF DATA.

STRICT JSON FORMAT – NO EXCEPTIONS:  
- OUTPUT MUST BE VALID JSON.  
- NO MARKDOWN, NO EXPLANATIONS, NO HEADERS.  

FIELDS (STRICTLY NOTHING ELSE):  
- "City" (string; city in the US, always provided in capital letters)
- "Company Name or Owners's Name" (string)  
- "Company Type" (string, ONLY OPTIONS : 'Attorney:', 'Real Estate:', 'Land Companies:', "Abstract Companies:","Abstractors:","Abstracts:", "Land Script:")
- "building" (string; if the address is a building add it here too)   
- "address" (string; full address if possible, otherwise partial) 
 

NON-NEGOTIABLE RULES:  
1. **EXTRACT EXACTLY AS WRITTEN. NO MODERNIZA

## AI model
Using gemini ai model I feed the pdf pages and the prompt.
The process is divided in 3 steps:


1.   Transforming image to json. Feeding one of the pre-processed images to the AI model and using the prompt, the model returns a raw response text. This text is cleaned of leading markdown words and potential brachet errors.
2.   Contactenation of json files. Taking into account each file stars and ends with[], a function deals with concatenation of each json string.
3.  Json do dictionary items inside of a lits, to be transformed into a dataframe format



In [ ]:
# OpenAI API setup (replace with your key)
from google.colab import userdata

# Replace with your actual API key
api_key = userdata.get('GOOGLE_API')

genai.configure(api_key=api_key)


In [ ]:
# !pip install --upgrade google-generativeai   if needed

In [ ]:
#####
def image_to_json(image_path, prompt):
    """Transforms an image to a JSON string using Gemini."""
    try:
        image = Image.open(image_path)
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format=image.format)
        image_bytes = img_byte_arr.getvalue()
        client = genai.GenerativeModel('gemini-1.5-flash')
        response = client.generate_content(
            contents=[
                {
                    "role": "user",
                    "parts": [{"text": prompt}, {"inline_data": {"mime_type": f"image/{image.format.lower()}", "data": image_bytes}}],
                }
            ]
        )

        # This part here is very unflexible. will see if it works for future files
        response.text
        raw_response = cleaning_json_ai(response.text)

        # POST CORRECTION AFTER AI CLEANING
        cleaned_text = raw_response.strip()
        cleaned_text = re.sub(r'^```(?:json)?\s*', '', cleaned_text)
        cleaned_text = re.sub(r'\s*```$', '', cleaned_text)
        # --- missing address values ---
        cleaned_text = re.sub(r'"address":\s*,(\s*"[^"]+":)', r'"address": null,\1', cleaned_text)
        cleaned_text = re.sub(r'"address":\s*,(\s*})', r'"address": null\1', cleaned_text)
        # ----missing Company Type values ---
        cleaned_text = re.sub(r'"Company Type":\s*,(\s*"[^"]+":)', r'"Company Type": null,\1', cleaned_text)
        cleaned_text = re.sub(r'"Company Type":\s*,(\s*})', r'"Company Type": null\1', cleaned_text)
        cleaned_text = re.sub(r'"Company Type":\s*"Real,\s*\n', r'"Company Type": "Real",\n},', cleaned_text)
        cleaned_text = re.sub(r'"Company Type":\s*"Real,\s*\n\s*\{', r'"Company Type": "Real",\n},', cleaned_text)
        cleaned_text = re.sub(r'"Company Type":\s*"Real,\s*$', r'"Company Type": "Real"\n}', cleaned_text)

        # ---- Missing Company Name/Owners values ---
        cleaned_text = re.sub(r'"Company Name or Owners\'s Name":\s*,(\s*"[^"]+":)', r'"Company Name or Owners\'s Name": null,\1', cleaned_text)
        cleaned_text = re.sub(r'"Company Name or Owners\'s Name":\s*,(\s*})', r'"Company Name or Owners\'s Name": null\1', cleaned_text)


        # ----missing building values  ---
        cleaned_text = re.sub(r'"building":\s*,(\s*"[^"]+":)', r'"building": null,\1', cleaned_text)
        cleaned_text = re.sub(r'"building":\s*,(\s*})', r'"building": null\1', cleaned_text)


        # correct if square brachets are remained open
        open_bracket_count = cleaned_text.count('[')
        close_bracket_count = cleaned_text.count(']')

        if open_bracket_count > close_bracket_count:
          cleaned_text += ']' * (open_bracket_count - close_bracket_count)


        # cleaned_text = re.sub(r'\{[^}]*?\n\s*\{', r'\{', cleaned_text)
        cleaned_text = re.sub(r'\,\]', r'}]', cleaned_text)
        cleaned_text = re.sub(r'}{', r'}, {', cleaned_text)

        return cleaned_text


    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

####
def concatenate_json_responses(json_response_list):
    """
    Concatenates a list of JSON response strings (where each string
    contains a JSON array), handling the surrounding "[]".
    """
    all_pages_content = "" # empty string to concatenate all responses
    first_page = True

    for json_response in json_response_list:
        page_content = json_response.strip()
        # if the page content starts with "[ ", take page content from the secont item onwards
        if page_content.startswith('['):
            page_content = page_content[1:]
        # if page content ends with "]" strip the last item
        if page_content.endswith(']'):
            page_content = page_content[:-1]

        ## to add a comma between results, if is not the first page and page_content is not empty
        # page_content is appended to all_pages_content preceded by a comma

        if not first_page and page_content:
            all_pages_content += ',' + page_content
        # if it is the first page it is added without the preceding comma, and first_page is set to FALSE
        elif first_page:
            all_pages_content += page_content
            first_page = False

    return "[" + all_pages_content + "]"

# READING THE PROMP2 :

prompt_file_path = os.path.join(cwd, "prompt2.txt")
# Read the prompt from the file
def read_prompt(prompt_file_path):
  try:
      with open(prompt_file_path, "r") as f:
          prompt = f.read().strip()
      print(f"Loaded prompt from: {prompt_file_path}")
      print(f"Prompt content: {prompt}")
  except FileNotFoundError:
      print(f"Error: Prompt file not found at {prompt_file_path}")
      exit()
  return prompt

prompt2 = read_prompt(prompt_file_path)

# SEcond prompt for cleaning format

def cleaning_json_ai(pages_json):
  client = genai.GenerativeModel('gemini-1.5-flash')
  response = client.generate_content(
          contents=[
            {
                "role": "user",
                "parts": [{"text": prompt2 + "\n\n" + pages_json}],
            }
        ]
    )
  return response.text

#
def json_to_data(json_string_list):
    """ This Function transforms a list of JSON strings to a list of dictionaries."""
    all_data=  []
    try:
      extracted_data = json.loads(json_string_list)
      # transforms in case of a dictionary to a list containing dictionaries
      if isinstance(extracted_data, dict):
          extracted_data = [extracted_data]
          for item in extracted_data:
            city = item.get("City")
            all_data.append({
                "City": city.strip() if city else None,
                "Company Name or Owners's Name": item.get("Company Name or Owners's Name"),
                "Company Type": item.get("Company Type"),
                "building": item.get("building"),
                "address": item.get("address")
          })
      return extracted_data

    except json.JSONDecodeError as e:
      print(f"Error decoding JSON string: {e}")
      return json_string_list

Loaded prompt from: /content/drive/MyDrive/2_FEINA/10_HU/01_TASKS/01_DIRECTORIES/prompt2.txt
Prompt content: YOU ARE AN EXPERT HISTORIAN SPECIALIZING IN EXTRACTING BUSINESS INFORMATION FROM HISTORICAL DOCUMENTS AND PRESENTING IT IN A PERFECTLY STRUCTURED JSON FORMAT.

YOUR ABSOLUTE PRIORITY IS TO GENERATE **VALID AND PARSABLE JSON**. ANY DEVIATION FROM PERFECT JSON SYNTAX IS UNACCEPTABLE AND WILL CAUSE FATAL ERRORS IN SUBSEQUENT PROCESSING.

YOUR TASK IS TO PROCESS THE FOLLOWING TEXT AND EXTRACT INFORMATION TO CREATE A JSON ARRAY OF BUSINESS ENTITIES. EACH ENTITY MUST ADHERE STRICTLY TO THE SPECIFIED FIELDS AND RULES.

FIELDS:
  - "City" (string; city in the US, ALWAYS PROVIDED IN CAPITAL LETTERS)
  - "Company Name or Owners's Name" (string)
  - "Company Type" (string, ONLY ONE OF THE FOLLOWING OPTIONS: 'Attorney:', 'Real Estate:', 'Land Companies:', "Abstract Companies:","Abstractors:","Abstracts:", "Land Script:")
  - "building" (string; name of the building if the address refers to 

In [ ]:

all_json_responses = []
for filename in os.listdir(processed_directory):
    if filename.lower().startswith('pre_processed') and filename.lower().endswith(
            '.jpg'):
        image_path = os.path.join(processed_directory, filename)
        # print(f"Processing image: {filename}")

        json_response = image_to_json(image_path, prompt)
        if json_response:
            # print(f"JSON Response for {filename}:\n{json_response}")
            all_json_responses.append(json_response)  # Append the JSON string to the list
            all_pages_content = concatenate_json_responses(all_json_responses)


## Manual correction.
Although the programme captures most of mistakes.

I have manually modified those in those instances. Most of the files run without any error although sometimes, mostly for long files, I have to correct one or two open brachets or missplaced commas. So it does not demand much extra time. Though it would be great to avoid mannually changing the json content, I have not yet found the right way.

In [ ]:
all_data = json_to_data(all_pages_content)
all_data

[{'City': 'HOUSTON',
  "Company Name or Owners's Name": 'J. H. Covert, Abst.',
  'Company Type': 'Abstractors:',
  'building': None,
  'address': None},
 {'City': 'HOUSTON',
  "Company Name or Owners's Name": 'Dooley & Hiete',
  'Company Type': 'Attorney:',
  'building': None,
  'address': None},
 {'City': 'HOUSTON',
  "Company Name or Owners's Name": 'McBride & Bell',
  'Company Type': 'Attorney:',
  'building': None,
  'address': None},
 {'City': 'HOUSTON',
  "Company Name or Owners's Name": 'Texas Co. Bank',
  'Company Type': 'Attorney:',
  'building': None,
  'address': None},
 {'City': 'HOUSTONIA',
  "Company Name or Owners's Name": 'W. R. Alexander',
  'Company Type': 'Attorney:',
  'building': None,
  'address': None},
 {'City': 'HUMANSVILLE',
  "Company Name or Owners's Name": 'Coffin & Rece',
  'Company Type': 'Attorney:',
  'building': None,
  'address': None},
 {'City': 'HUMANSVILLE',
  "Company Name or Owners's Name": 'Evans & Waldron',
  'Company Type': 'Attorney:',
  'bui

In [ ]:
df = all_data.copy()
type(df)


list

In [ ]:
# tranforming list of dictionaries to a nested dictionary grouping info by company name.
transformed_dict = {
    # filter out empty company names
    entry["Company Name or Owners's Name"]: {
        "City": entry.get("City"),
        "Company Type": entry.get("Company Type"),
        "building": entry.get("building"),
        "address": entry.get("address")
    }
    for entry in df
    if entry.get("Company Name or Owners's Name")  # Filter out None or empty strings
}
# transformed_dict

In [ ]:
df2 = pd.DataFrame(transformed_dict)
df2

,"J. H. Covert, Abst.",Dooley & Hiete,McBride & Bell,Texas Co. Bank,W. R. Alexander,Coffin & Rece,Evans & Waldron,Missouri Land Co.,Wayts & Beadles,Wilson & Bloomfield,...,A. G. Hoppock,Laclede County Abst. Co.,Lebanon Realty Co.,W. W. Wellman,O. J. Ratliff,S. B. Bowin,Bowyer & Snider,Todd M. George,Kimball & Yankee,Land & Depuy
City,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTONIA,HUMANSVILLE,HUMANSVILLE,HUMANSVILLE,HUME,HUME,...,LEBANON,LEBANON,LEBANON,LEBANON,LEEDS,LEE'S SUMMIT,LEE'S SUMMIT,LEE'S SUMMIT,LEE'S SUMMIT,LEE'S SUMMIT
Company Type,Abstractors:,Attorney:,Attorney:,Attorney:,Attorney:,Attorney:,Attorney:,Land Companies:,Land Companies:,Land Companies:,...,Real Estate:,Real Estate:,Real Estate:,Real Estate:,Real Estate:,Real Estate:,Real Estate:,Real Estate:,Real Estate:,Real Estate:
building,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
address,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# Transpose df2
df2_transposed = df2.T

# Rename columns
df2_t = df2_transposed.rename(
    columns={
        "City": "City",
        "Company Type": "Company type",
        "building": "building",
        "address": "address"
    }
)

# Rename the index to "Company Name"
df2_t.index.name = "Company Name or Owners's Name"

# Reset index to make "Company Name" a regular column
df2_t = df2_t.reset_index()

# Display the transposed DataFrame
df2_t

,Company Name or Owners's Name,City,Company type,building,address
0,"J. H. Covert, Abst.",HOUSTON,Abstractors:,None,None
1,Dooley & Hiete,HOUSTON,Attorney:,None,None
2,McBride & Bell,HOUSTON,Attorney:,None,None
3,Texas Co. Bank,HOUSTON,Attorney:,None,None
4,W. R. Alexander,HOUSTONIA,Attorney:,None,None
...,...,...,...,...,...
851,S. B. Bowin,LEE'S SUMMIT,Real Estate:,None,None
852,Bowyer & Snider,LEE'S SUMMIT,Real Estate:,None,None
853,Todd M. George,LEE'S SUMMIT,Real Estate:,None,None
854,Kimball & Yankee,LEE'S SUMMIT,Real Estate:,None,None


In [ ]:
# SLICING DATA - city must be equal to city
# city = "Atlanta"
df2_t['City'] = df2_t['City'].str.title()

# df2_t
df2_cleaned = df2_t[df2_t["City"] == city].reset_index(drop=True)





df2_cleaned

,Company Name or Owners's Name,City,Company type,building,address
0,Independence Inv. Co.,Kansas City,Real Estate:,None,222 W. Lexington
1,"JNO. A, EATON",Kansas City,Attorney:,300 U. S. & Mex. Trust bldg.,None
2,S. A. AKINS & CO.,Kansas City,Real Estate:,325 Midland bldg.,None
3,ALLISON & RITCHIE LAND CO.,Kansas City,Land Companies:,None,Union Depot
4,BAIRD & HUSELTON,Kansas City,Real Estate:,Junction bldg.,None
...,...,...,...,...,...
646,J. A. Wright,Kansas City,Real Estate:,Shukert bldg.,
647,L. R. Wright,Kansas City,Real Estate:,R. A. Long bidg.,
648,M. B Wright,Kansas City,Real Estate:,,919 Baltimore ave.
649,James R. Young,Kansas City,Real Estate:,Ridge bldg.,


In [ ]:
def full_address(df, city, state, country):
  df['full_address'] = ''

  # clean empty values and replace them for na
  df['building'] = df['building'].replace(r'^\s*$', np.nan, regex=True)
  df['address'] = df['address'].replace(r'^\s*$', np.nan, regex=True)


  for index, value in df.iterrows():
    # When both building and address are present
    if pd.notna(value['building']) and pd.notna(value['address']):
      df.loc[index, 'full_address'] = value['address'] + ', ' + value['building'] + ', ' +  city + ', '+ state + ', ' + country
    # Only building
    elif pd.notna(value['building']):
      df.loc[index, 'full_address'] = value['building'] + ', ' +  city + ', '+ state + ', ' + country
    # only address
    elif pd.notna(value['address']):
      df.loc[index, 'full_address'] = value['address'] + ', ' +  city + ', '+ state + ', ' + country
    else:
      df.loc[index, 'full_address'] = ''
  # Final cleanup: remove any leftover parentheses and strip spaces
  df['full_address'] = df['full_address'].str.strip().str.replace(r'\(.*?\)', '', regex=True)

  return df


city_fulladr = full_address(df2_cleaned, city = city, state = state, country = country)
city_fulladr

city_fulladr.to_csv(f"ai_files/{city}/ai_data_{city}.csv", index=False)

def geocoding(df):
  geolocator = Nominatim(user_agent="geo_locator")  # Initialize Nominatim geocoder
  geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)  # Add rate limiting to avoid overloading the service

  for index, row in df.iterrows():
    if pd.notna(row['full_address']) and isinstance(row['full_address'], str) and 'nan' not in row['full_address']:
      location = geocode(row["full_address"])
      if location:
        df.loc[index, "Lat"] = location.latitude # create latitud column
        df.loc[index, "Lon"] = location.longitude # longitude column

      else:
        df.loc[index, "Lat"] = None
        df.loc[index, "Lon"] = None

    else:
      df.loc[index, "Lat"] = None
      df.loc[index, "Lon"] = None

  return df

geo_city = geocoding(city_fulladr)
geo_city.to_csv(f"ai_files/{city}/geo_{city}.csv", index=False)

Streaming output truncated to the last 5000 lines.
  File "/usr/local/lib/python3.11/dist-packages/geopy/extra/rate_limiter.py", line 136, in _retries_gen
    yield i  # Run the function.
    ^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/geopy/extra/rate_limiter.py", line 274, in __call__
    res = self.func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/geopy/geocoders/nominatim.py", line 297, in geocode
    return self._call_geocoder(url, callback, timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/geopy/geocoders/base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/geopy/adapters.py", line 472, in get_json
    resp = self._request(url, timeout=timeout, heade

In [ ]:
geo_city

,Company Name or Owners's Name,City,Company type,building,address,full_address,Lat,Lon
0,Independence Inv. Co.,Kansas City,Real Estate:,None,222 W. Lexington,"222 W. Lexington, Kansas City, Missouri, USA",39.111368,-94.559172
1,"JNO. A, EATON",Kansas City,Attorney:,300 U. S. & Mex. Trust bldg.,None,"300 U. S. & Mex. Trust bldg., Kansas City, Mis...",NaN,NaN
2,S. A. AKINS & CO.,Kansas City,Real Estate:,325 Midland bldg.,None,"325 Midland bldg., Kansas City, Missouri, USA",NaN,NaN
3,ALLISON & RITCHIE LAND CO.,Kansas City,Land Companies:,None,Union Depot,"Union Depot, Kansas City, Missouri, USA",NaN,NaN
4,BAIRD & HUSELTON,Kansas City,Real Estate:,Junction bldg.,None,"Junction bldg., Kansas City, Missouri, USA",NaN,NaN
...,...,...,...,...,...,...,...,...
646,J. A. Wright,Kansas City,Real Estate:,Shukert bldg.,NaN,"Shukert bldg., Kansas City, Missouri, USA",NaN,NaN
647,L. R. Wright,Kansas City,Real Estate:,R. A. Long bidg.,NaN,"R. A. Long bidg., Kansas City, Missouri, USA",NaN,NaN
648,M. B Wright,Kansas City,Real Estate:,NaN,919 Baltimore ave.,"919 Baltimore ave., Kansas City, Missouri, USA",39.103268,-94.584067
649,James R. Young,Kansas City,Real Estate:,Ridge bldg.,NaN,"Ridge bldg., Kansas City, Missouri, USA",NaN,NaN


In [ ]:
geo_city.to_csv(f"ai_files/{city}/geo_{city}.csv", index=False)